In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
%pwd

'/mnt/c/Users/mtr22/code/mre-pinn/notebooks'

In [2]:
import sys, os
import numpy as np
import xarray as xr
import torch
import matplotlib.pyplot as plt

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

/home/mtr22/anaconda3/envs/MRE-PINN2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch

Loading /mnt/c/Users/mtr22/code/mre-pinn/notebooks/../mre_pinn/__init__.py


In [3]:
# load the MATLAB file
data_root = '../data/BIOQIC'
mat_base = 'four_target_phantom.mat'

mat_file = data_root + '/' + mat_base
data, rev_axes = mre_pinn.data.load_mat_data(mat_file, verbose=True)

Loading ../data/BIOQIC/four_target_phantom.mat
    __header__: <class 'bytes'>
    __version__: <class 'str'>
    __globals__: <class 'list'>
    u_ft: <class 'numpy.ndarray'> (100, 80, 10, 3, 6) complex128


In [4]:
# convert to xarray
u_array = data['u_ft'].T
u_dims = ['frequency', 'component', 'z', 'x', 'y']
u_coords = {
    'frequency': 50 + np.arange(u_array.shape[0]) * 10,
    'x': np.arange(u_array.shape[3]) * 1, # mm
    'y': np.arange(u_array.shape[4]) * 1, # mm
    'z': np.arange(u_array.shape[2]) * 1, # mm
    'component': ['y', 'x', 'z'],
}
u_array = xr.DataArray(u_array, dims=u_dims, coords=u_coords)
u_array = u_array.transpose('frequency', 'x', 'y', 'z', 'component')
u_array

<xarray.DataArray (frequency: 6, x: 80, y: 100, z: 10, component: 3)>
array([[[[[ 1.80392877e-03-2.21015603e-03j,
            4.41610136e-12+4.15293497e-13j,
            7.43590828e-12+9.91063094e-12j],
          [ 1.77340791e-03-2.24933157e-03j,
            1.31072889e-21-1.55283972e-22j,
            2.72998684e-20+3.60913412e-20j],
          [ 1.73815892e-03-2.29137902e-03j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          ...,
          [ 1.53102016e-03-2.50421953e-03j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          [ 1.48959832e-03-2.53624011e-03j,
            1.31146067e-12-2.80054899e-13j,
            1.07067832e-11+1.17379437e-11j],
          [ 1.44979279e-03-2.56093656e-03j,
            0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j]],

...

         [[-1.00023484e-04+1.36753538e-04j,
            1.20711900e-04+5.54074597e-05j,
           -1.96984648e-03+6.47758301e-04j],
          [-9.95064987e-05+1.46854257e-04j,
            1.26588107e-04+2.97468078e-05j,
           -1.99102903e-03+6.01061728e-04j],
          [-9.50972761e-05+1.22089349e-04j,
            1.09840310e-04+3.24128881e-05j,
           -1.91212253e-03+6.23055927e-04j],
          ...,
          [ 2.95336889e-05+2.89242094e-05j,
           -9.08173401e-05+1.84582541e-04j,
           -1.98020856e-03+5.95482056e-04j],
          [ 2.40140120e-05+3.34154881e-05j,
           -9.65727975e-05+1.50526975e-04j,
           -1.95558998e-03+6.29691257e-04j],
          [ 1.03669066e-05+5.69770862e-05j,
           -7.38893222e-05+8.16353244e-05j,
           -1.93945465e-03+6.69014975e-04j]]]]])
Coordinates:
  * frequency  (frequency) int64 50 60 70 80 90 100
  * x          (x) int64 0 1 2 3 4 5 6 7 8 9 ... 70 71 72 73 74 75 76 77 78 79
  * y          (y) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98 99
  * z          (z) int64 0 1 2 3 4 5 6 7 8 9
  * component  (component) <U1 'y' 'x' 'z'

In [40]:
%autoreload

# convert to point set boundary condition
u_bc = mre_pinn.data.NDArrayBC(u_array, batch_size=128)
u_bc.points.shape, u_bc.values.shape

((480000, 4), torch.Size([480000, 3]))

In [41]:
%autoreload

# set up PDE with geometry and boundary conditions
pde = mre_pinn.pde.HelmholtzPDE()
geometry = deepxde.geometry.Hypercube(u_bc.points.min(axis=0), u_bc.points.max(axis=0))
pde = deepxde.data.PDE(geometry, pde, u_bc, num_domain=128)
pde.train_x.shape

(256, 4)

In [42]:
%autoreload

# initialize neural network
net = mre_pinn.model.Parallel([
    mre_pinn.model.ComplexFFN(
        n_input=4,
        n_layers=5,
        n_hidden=16,
        n_output=n_output,
        activ_fn=torch.sin,
        w0=32
    ) for n_output in [3, 1] # u and mu
])

# standardize inputs to [-1, 1]
x = u_bc.points
x_loc = np.mean(x, axis=0)
x_scale = (np.max(x, axis=0) - np.min(x, axis=0)) / 2

# normalize outputs
u = u_bc.values.cpu().numpy()
u = np.concatenate([u.real, u.imag], axis=1)
u_loc = np.mean(u, axis=0)
u_scale = np.std(u, axis=0)

# initialize model weights
net[0].init_weights(input_loc=x_loc, input_scale=x_scale, output_scale=u_scale, output_loc=u_loc)
net[1].init_weights(input_loc=x_loc, input_scale=x_scale, output_scale=1, output_loc=0)

net

Parallel(
  (0): ComplexFFN(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): Linear(in_features=20, out_features=16, bias=True)
    (2): Linear(in_features=36, out_features=16, bias=True)
    (3): Linear(in_features=52, out_features=16, bias=True)
    (4): Linear(in_features=68, out_features=6, bias=True)
  )
  (1): ComplexFFN(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): Linear(in_features=20, out_features=16, bias=True)
    (2): Linear(in_features=36, out_features=16, bias=True)
    (3): Linear(in_features=52, out_features=16, bias=True)
    (4): Linear(in_features=68, out_features=2, bias=True)
  )
)

In [ ]:
def msae_loss(y_true, y_pred):
    '''
    Mean squared absolute error.
    '''
    return torch.mean(torch.abs(y_true - y_pred)**2)

model = deepxde.Model(pde, net)
model.compile(
    optimizer='adam',
    lr=1e-4,
    loss=msae_loss,
    loss_weights=[1e-6, 1]
)
model.train(100000, display_every=1000)

Compiling model...
'compile' took 0.000174 s

Training model...

Step      Train loss              Test loss               Test metric
0         [7.80e-06, 3.71e-05]    [7.80e-06, 3.71e-05]    []  
1000      [5.47e-07, 1.69e-05]    [5.47e-07, 1.69e-05]    []  


In [44]:
import matplotlib as mpl

w_map = mre_pinn.visual.wave_color_map()
w_max = 0.005
wave_kws = dict(cmap=w_map, vmin=-w_max, vmax=w_max)

e_map = mre_pinn.visual.elast_color_map()
e_max = 100
elast_kws = dict(cmap=e_map, vmin=0, vmax=e_max)


class TrainingPlot(deepxde.display.TrainingDisplay):
    
    def __init__(self):
        self.initialized = False
    
    def initialize(self):
        self.fig, self.axes = plt.subplots(1, 2, figsize=(8,4), squeeze=False)
        labels = ['PDE loss', 'data loss']
        self.lines = [self.axes[0,0].plot([], [], label=labels[i])[0] for i in range(2)]
        self.scatter = self.axes[0,1].scatter([], [], marker='o', s=0.2, cmap=w_map, vmin=-w_max, vmax=w_max)
        self.axes[0,0].legend(frameon=False)
        self.axes[0,0].set_yscale('log')
        self.axes[0,0].grid(color='gray', linestyle=':')
        self.axes[0,1].set_xlim(-0.5, 0.5)
        self.axes[0,1].set_ylim(-0.5, 0.5)
        self.initialized = True

    def __call__(self, train_state):

        if not self.initialized:
            self.initialize()
        
        for i, line in enumerate(self.lines):
            new_x = train_state.step
            new_y = train_state.loss_train[i]
            line.set_xdata(np.append(line.get_xdata(), new_x))
            line.set_ydata(np.append(line.get_ydata(), new_y))
            
        if (train_state.step % 10) == 0:
            x = train_state.X_train
            ux = train_state.y_pred_train[:,0]
            uy = train_state.y_pred_train[:,1]
            mu = train_state.y_pred_train[:,2]

            #self.scatter.set_offsets(x)
            #w_min = np.min(ux.real)
            #w_max = np.max(ux.real)
            #w_max = max(abs(w_min), abs(w_max))
            #n = mpl.colors.Normalize(vmin=-w_max, vmax=w_max)
            #m = mpl.cm.ScalarMappable(norm=n, cmap=w_map)
            #self.scatter.set_facecolor(m.to_rgba(ux.real))
            
        self.axes[0,0].relim()
        self.axes[0,0].autoscale_view()
    
        self.fig.canvas.draw()

In [ ]:
%autoreload

deepxde.display.training_display = TrainingPlot()
try:
    model.train(20000, display_every=10)
except KeyboardInterrupt as e:
    print('Interrupt', file=sys.stderr)

In [ ]:
torch.save(net.state_dict(), 'wave_sim.model')

In [ ]:
x = wave_image.points
ux_true = wave_image.values[:,0].cpu().numpy()
uy_true = wave_image.values[:,1].cpu().numpy()
mu_true = elast_image.values.cpu().numpy()

outputs = model.predict(x)
ux_pred = outputs[:,0]
uy_pred = outputs[:,1]
mu_pred = outputs[:,2]

# configure plot grid
n_rows = 6
n_cols = 3
size = 2
hspace = 0.2
wspace = 0.4
lpad = 1.1
rpad = 0.2
bpad = 0.6
tpad = 0.4
fig_height = n_rows * size + (n_rows - 1) * hspace + bpad + tpad
fig_width = (n_cols - 1) * (size + wspace) + size/8 + lpad + rpad
fig, axes = plt.subplots(
    n_rows, n_cols, squeeze=False,
    figsize=(fig_width, fig_height),
    gridspec_kw=dict(
        width_ratios=[1/8,1,1],
        hspace=hspace, wspace=wspace,
        left=lpad/fig_width, right=1 - rpad/fig_width,
        bottom=bpad/fig_height, top=1 - tpad/fig_height
    )
)

# header labels
axes[0,1].set_title('true')
axes[0,2].set_title('predicted')

# plot colorbar, true, and predicted for each variable
sc = plot_points_2d(x, ux_true.real, [1, 1], ax=axes[0,1], **wave_kws)
sc = plot_points_2d(x, ux_pred.real, [1, 1], ax=axes[0,2], **wave_kws)
plot_colorbar(sc, ax=axes[0,0], label='Re[$u_x$]')

sc = plot_points_2d(x, ux_true.imag, [1, 1], ax=axes[1,1], **wave_kws)
sc = plot_points_2d(x, ux_pred.imag, [1, 1], ax=axes[1,2], **wave_kws)
plot_colorbar(sc, ax=axes[1,0], label='Im[$u_x$]')

sc = plot_points_2d(x, uy_true.real, [1, 1], ax=axes[2,1], **wave_kws)
sc = plot_points_2d(x, uy_pred.real, [1, 1], ax=axes[2,2], **wave_kws)
plot_colorbar(sc, ax=axes[2,0], label='Re[$u_y$]')

sc = plot_points_2d(x, uy_true.imag, [1, 1], ax=axes[3,1], **wave_kws)
sc = plot_points_2d(x, uy_pred.imag, [1, 1], ax=axes[3,2], **wave_kws)
plot_colorbar(sc, ax=axes[3,0], label='Im[$u_y$]')

sc = plot_points_2d(x, mu_true.real, [1, 1], ax=axes[4,1], **elast_kws)
sc = plot_points_2d(x, mu_pred.real, [1, 1], ax=axes[4,2], **elast_kws)
plot_colorbar(sc, ax=axes[4,0], label='Re[$\mu$]')

sc = plot_points_2d(x, mu_true.imag, [1, 1], ax=axes[5,1], **elast_kws)
sc = plot_points_2d(x, mu_pred.imag, [1, 1], ax=axes[5,2], **elast_kws)
plot_colorbar(sc, ax=axes[5,0], label='Im[$\mu$]')

In [ ]:
u = np.load('../data/wave_sim/steady_state_wave.npy')

ux = u[...,0]
uy = u[...,1]
dx = 0.005

ux_x, ux_y = np.gradient(ux, dx)
ux_xx, ux_xy = np.gradient(ux_x, dx)
ux_yx, ux_yy = np.gradient(ux_y, dx)
laplace_ux = ux_xx + ux_yy

rho = 1.0
omega = 10*(2*np.pi)
mu = -rho * omega**2 * ux / laplace_ux
mu.shape

fig, ax = plt.subplots()
plot_image_2d(mu.real, [1, 1], ax, **elast_kws)